# Transformer-XL: Attentive Language Models Beyond a Fixed-Length Context

## Introduction
为了帮助理解XLNet，本文对其核心框架Transformer-XL作一个解读。本文发表在ACL2019上，论文想要解决的问题：如何赋予编码器捕获长距离依赖的能力。目前在自然语言处理领域，Transformer的编码能力超越了RNN，但是对长距离依赖的建模能力仍然不足。在基于LSTM的模型中，为了建模长距离依赖，提出了门控机制和梯度裁剪，目前可以编码的最长距离在200左右[5]。在基于Transformer的模型中，允许词之间直接建立联系【self-attention】，能够更好地捕获长期依赖关系，但是还是有限制。
# Motivation
Transformer编码固定长度的上下文，即将一个长的文本序列截断为几百个字符的固定长度片段(segment)，然后分别编码每个片段[1]，片段之间没有任何的信息交互。比如BERT，序列长度的极限一般在512。动机总结如下：

+ Transformer无法建模超过固定长度的依赖关系，对长文本编码效果差。
+ Transformer把要处理的文本分割成等长的片段，通常不考虑句子（语义）边界，导致上下文碎片化(context fragmentation)。通俗来讲，一个完整的句子在分割后，一半在前面的片段，一半在后面的片段。
文章围绕如何建模长距离依赖，提出Transformer-XL【XL是extra long的意思】：
+ 提出片段级递归机制(segment-level recurrence mechanism)，引入一个记忆(memory)模块（类似于cache或cell），循环用来建模片段之间的联系。
  1. 使得长距离依赖的建模成为可能；
  2. 使得片段之间产生交互，解决上下文碎片化问题。
+ 提出相对位置编码机制(relative position embedding scheme)，代替绝对位置编码。
  1. 在memory的循环计算过程中，避免时序混淆【见model部分】，位置编码可重用。
小结一下，片段级递归机制为了解决编码长距离依赖和上下文碎片化，相对位置编码机制为了实现片段级递归机制而提出，解决可能出现的时序混淆问题。

This is an implementation of Transformer-XL: Attentive Language Models Beyond a Fixed-Length Context in PyTorch.

Transformer has a limited attention span, equal to the length of the sequence trained in parallel. All these positions have a fixed positional encoding. Transformer XL increases this attention span by letting each of the positions pay attention to precalculated past embeddings. For instance if the context length is l, it will keep the embeddings of all layers for previous batch of length l and feed them to current step. If we use fixed-positional encodings these pre-calculated embeddings will have the same positions as the current context. They introduce relative positional encoding, where the positional encodings are introduced at the attention calculation.

Annotated implementation of relative multi-headed attention is in relative_mha.py .

Here's the training code and a notebook for training a transformer XL model on Tiny Shakespeare dataset.

In [ ]:
from typing import List,Optional

import torch
import torch.nn as nn

from labml_helpers.module import Module
from labml_nn.utils import clone_module_list
from .relative_mha import RelativeMultiHeadAttention
from

# Transformer XL Layer
The transformer XL model comprises of a number of these layers.

In [ ]:
class TransformerXLLayer(Module):
